# Task 2: Text Chunking, Embedding, and Vector Store Indexing

This notebook completes Task 2 by:
1. Loading the filtered complaints data
2. Chunking narratives into smaller pieces
3. Generating embeddings using sentence-transformers
4. Creating a FAISS vector store for efficient retrieval
5. Testing the vector store with similarity search
6. Providing a summary for reporting

**Prerequisites:** Task 1 must be completed (filtered_complaints.csv must exist)

## Step 1: Setup and Load Data

In [1]:
import pandas as pd
import numpy as np
import os
import time

# Paths
input_file = '../data/processed/filtered_complaints.csv'
chunks_file = '../data/processed/complaint_chunks.csv'

# Verify input file
if not os.path.exists(input_file):
    raise FileNotFoundError(f"File not found at: {os.path.abspath(input_file)}")

# Load filtered dataset
print("Loading filtered complaints...")
df = pd.read_csv(input_file)
print(f"Loaded {len(df):,} complaints with columns: {df.columns.tolist()}")

# Verify required columns
required_columns = ['Complaint ID', 'Product', 'Consumer complaint narrative']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"Missing columns: {missing_columns}")

print("✅ Data loaded successfully!")

Loading filtered complaints...
Loaded 355,635 complaints with columns: ['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue', 'Consumer complaint narrative', 'Company public response', 'Company', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?', 'Submitted via', 'Date sent to company', 'Company response to consumer', 'Timely response?', 'Consumer disputed?', 'Complaint ID']
✅ Data loaded successfully!


## Step 2: Chunk Narratives

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize text splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Based on Task 1 narrative length analysis
    chunk_overlap=50,
    length_function=lambda x: len(x.split())
)

# Split narratives
print("Chunking narratives...")
chunks = []
for idx, row in df.iterrows():
    splits = splitter.split_text(row['Consumer complaint narrative'])
    for split in splits:
        chunks.append({
            'complaint_id': row['Complaint ID'],
            'product': row['Product'],
            'chunk': split
        })

# Save chunks
df_chunks = pd.DataFrame(chunks)
os.makedirs('../data/processed', exist_ok=True)
df_chunks.to_csv(chunks_file, index=False)
print(f"✅ Created {len(df_chunks):,} chunks, saved to {chunks_file}")

# Display chunk statistics
chunk_lengths = df_chunks['chunk'].str.split().str.len()
print(f"\n📊 Chunk Statistics:")
print(f"Total chunks: {len(df_chunks):,}")
print(f"Unique complaints: {df_chunks['complaint_id'].nunique():,}")
print(f"Average words per chunk: {chunk_lengths.mean():.1f}")
print(f"Median words per chunk: {chunk_lengths.median():.1f}")
print(f"Products: {df_chunks['product'].unique()}")

Chunking narratives...
✅ Created 392,406 chunks, saved to ../data/processed/complaint_chunks.csv

📊 Chunk Statistics:
Total chunks: 392,406
Unique complaints: 355,635
Average words per chunk: 184.8
Median words per chunk: 140.0
Products: ['Checking or savings account'
 'Money transfer, virtual currency, or money service'
 'Credit card or prepaid card' 'Consumer Loan']


## Step 3: Generate Embeddings

In [3]:
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

# Load chunks if not already loaded
if 'df_chunks' not in locals():
    df_chunks = pd.read_csv(chunks_file)

# Initialize model
print("🔧 Initializing embedding model...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"✅ Model loaded: {model.get_sentence_embedding_dimension()} dimensions")

# Test embedding on a small sample first
print("\n🧪 Testing embedding on sample...")
sample_texts = df_chunks['chunk'].head(5).tolist()
sample_embeddings = model.encode(sample_texts, batch_size=32, show_progress_bar=True)
print(f"✅ Sample embeddings shape: {sample_embeddings.shape}")

d:\Projects\Python\creditrust-complaint-analysis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔧 Initializing embedding model...
✅ Model loaded: 384 dimensions

🧪 Testing embedding on sample...


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.36it/s]

✅ Sample embeddings shape: (5, 384)


In [4]:
# Generate embeddings for all chunks
print("🚀 Generating embeddings for all chunks...")
print("This may take 15-45 minutes depending on your system...")

start_time = time.time()

# Process in batches to avoid memory issues
batch_size = 1000
all_embeddings = []
total_batches = (len(df_chunks) + batch_size - 1) // batch_size

for i in range(0, len(df_chunks), batch_size):
    batch_num = (i // batch_size) + 1
    batch = df_chunks['chunk'].iloc[i:i+batch_size].tolist()
    
    print(f"Processing batch {batch_num}/{total_batches} ({i+1}-{min(i+batch_size, len(df_chunks)):,} chunks)...")
    
    batch_embeddings = model.encode(batch, batch_size=32, show_progress_bar=False)
    all_embeddings.extend(batch_embeddings.tolist())

embedding_time = time.time() - start_time
print(f"\n✅ Generated embeddings in {embedding_time:.1f} seconds")
print(f"Total embeddings: {len(all_embeddings):,}")
print(f"Embedding dimension: {len(all_embeddings[0])}")

🚀 Generating embeddings for all chunks...
This may take 15-45 minutes depending on your system...
Processing batch 1/393 (1-1,000 chunks)...
Processing batch 2/393 (1001-2,000 chunks)...
Processing batch 3/393 (2001-3,000 chunks)...
Processing batch 4/393 (3001-4,000 chunks)...
Processing batch 5/393 (4001-5,000 chunks)...
Processing batch 6/393 (5001-6,000 chunks)...
Processing batch 7/393 (6001-7,000 chunks)...
Processing batch 8/393 (7001-8,000 chunks)...
Processing batch 9/393 (8001-9,000 chunks)...
Processing batch 10/393 (9001-10,000 chunks)...
Processing batch 11/393 (10001-11,000 chunks)...
Processing batch 12/393 (11001-12,000 chunks)...
Processing batch 13/393 (12001-13,000 chunks)...
Processing batch 14/393 (13001-14,000 chunks)...
Processing batch 15/393 (14001-15,000 chunks)...
Processing batch 16/393 (15001-16,000 chunks)...
Processing batch 17/393 (16001-17,000 chunks)...
Processing batch 18/393 (17001-18,000 chunks)...
Processing batch 19/393 (18001-19,000 chunks)...
Pr

## Step 4: Create FAISS Vector Store

In [5]:
import faiss

print("🏗️ Building FAISS vector store...")

# Convert embeddings to numpy array
embeddings_np = np.array(all_embeddings, dtype=np.float32)
print(f"Embeddings array shape: {embeddings_np.shape}")

# Create FAISS index
dimension = embeddings_np.shape[1]  # Should be 384 for all-MiniLM-L6-v2
print(f"Creating FAISS index with dimension: {dimension}")

# Use IndexFlatL2 for exact L2 distance search
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

print(f"✅ FAISS index created with {index.ntotal:,} vectors")
print(f"Index dimension: {index.d}")

🏗️ Building FAISS vector store...
Embeddings array shape: (392406, 384)
Creating FAISS index with dimension: 384
✅ FAISS index created with 392,406 vectors
Index dimension: 384


## Step 5: Save Vector Store and Metadata

In [6]:
# Create vector store directory
vector_store_dir = '../vector_store'
os.makedirs(vector_store_dir, exist_ok=True)

# Save FAISS index
index_file = os.path.join(vector_store_dir, 'faiss_index.bin')
print(f"💾 Saving FAISS index to {index_file}...")
faiss.write_index(index, index_file)
print(f"✅ FAISS index saved ({os.path.getsize(index_file) / (1024*1024):.1f} MB)")

# Save metadata
metadata_file = os.path.join(vector_store_dir, 'metadata.csv')
print(f"💾 Saving metadata to {metadata_file}...")
metadata_df = df_chunks[['complaint_id', 'product', 'chunk']].copy()
metadata_df.to_csv(metadata_file, index=False)
print(f"✅ Metadata saved ({os.path.getsize(metadata_file) / (1024*1024):.1f} MB)")

print(f"\n📁 Vector store files created in: {vector_store_dir}")

💾 Saving FAISS index to ../vector_store\faiss_index.bin...
✅ FAISS index saved (574.8 MB)
💾 Saving metadata to ../vector_store\metadata.csv...
✅ Metadata saved (398.1 MB)

📁 Vector store files created in: ../vector_store


## Step 6: Verify Vector Store

In [7]:
print("🔍 Verifying vector store...")

# Test loading FAISS index
print("Loading FAISS index...")
test_index = faiss.read_index(index_file)
print(f"✅ FAISS index loaded: {test_index.ntotal:,} vectors, dimension {test_index.d}")

# Test loading metadata
print("Loading metadata...")
test_metadata = pd.read_csv(metadata_file)
print(f"✅ Metadata loaded: {len(test_metadata):,} rows")
print(f"Metadata columns: {test_metadata.columns.tolist()}")

# Test similarity search
print("\n🧪 Testing similarity search...")
test_question = "What are common credit card issues?"
test_embedding = model.encode([test_question])
distances, indices = test_index.search(test_embedding, k=3)

print(f"Test question: {test_question}")
print(f"Retrieved {len(indices[0])} similar chunks:")
for i, (idx, distance) in enumerate(zip(indices[0], distances[0])):
    chunk_text = test_metadata.iloc[idx]['chunk'][:200] + "..."
    product = test_metadata.iloc[idx]['product']
    print(f"  {i+1}. [{product}] {chunk_text}")
    print(f"     Distance: {distance:.3f}")

print("\n🎉 Vector store verification successful!")

🔍 Verifying vector store...
Loading FAISS index...
✅ FAISS index loaded: 392,406 vectors, dimension 384
Loading metadata...
✅ Metadata loaded: 392,406 rows
Metadata columns: ['complaint_id', 'product', 'chunk']

🧪 Testing similarity search...
Test question: What are common credit card issues?
Retrieved 3 similar chunks:
  1. [Checking or savings account] general issues with debit card...
     Distance: 0.727
  2. [Credit card or prepaid card] as usual was using credit card i had fine established credit and company  xxxx  put freeze on uses of card...
     Distance: 0.780
  3. [Credit card or prepaid card] citizens credit card i paid my credit card off having problems for the last two months with payments being made and not being posted interest late fees being charged to my account then charging intere...
     Distance: 0.800

🎉 Vector store verification successful!


## Step 7: Test Similarity Search with Different Queries

In [8]:
# Test similarity search on the FAISS index
print("🔍 Testing similarity search with different queries...")

# Example queries
test_queries = [
    "Why are people unhappy with BNPL?",
    "What billing issues do customers report?",
    "What fraud-related complaints exist?",
    "What customer service problems are mentioned?"
]

for query in test_queries:
    print(f"\n📝 Query: {query}")
    
    # Generate embedding for query
    query_embedding = model.encode([query])
    
    # Search FAISS index
    k = 3
    distances, indices = test_index.search(np.array(query_embedding, dtype=np.float32), k)
    
    print(f"Top {k} most similar complaint chunks:")
    
    for i, idx in enumerate(indices[0]):
        row = test_metadata.iloc[idx]
        print(f"  {i+1}. [{row['product']}] Complaint {row['complaint_id']}")
        print(f"     Chunk: {row['chunk'][:150]}...")
        print(f"     Distance: {distances[0][i]:.4f}")
    
    print("-" * 80)

🔍 Testing similarity search with different queries...

📝 Query: Why are people unhappy with BNPL?
Top 3 most similar complaint chunks:
  1. [Checking or savings account] Complaint 9967256
     Chunk: have with bmo again i have never had such a miserable experience with a financial in
stitution at no point did anyone at bmo take ownership of my conc...
     Distance: 1.0844
  2. [Credit card or prepaid card] Complaint 6067188
     Chunk: i have spent well over 17000 worth of my time on this matter but to me there is a huge principle at stake here from the feelings i have gone through m...
     Distance: 1.1661
  3. [Checking or savings account] Complaint 9044009
     Chunk: as a customer i feel very disappointed in their unfairness and disrespectful treatment of me having me steadily reach out to no avail call and go in p...
     Distance: 1.1814
--------------------------------------------------------------------------------

📝 Query: What billing issues do customers report?
Top 3 most

## Step 8: Task 2 Summary and Reporting

In [9]:
print("📊 Task 2 Completion Summary")
print("=" * 50)
print(f"✅ Chunks processed: {len(df_chunks):,}")
print(f"✅ Embeddings generated: {len(all_embeddings):,}")
print(f"✅ Embedding dimension: {len(all_embeddings[0])}")
print(f"✅ FAISS vectors: {test_index.ntotal:,}")
print(f"✅ Processing time: {embedding_time:.1f} seconds")
print(f"✅ Vector store size: {os.path.getsize(index_file) / (1024*1024):.1f} MB")
print(f"✅ Metadata size: {os.path.getsize(metadata_file) / (1024*1024):.1f} MB")

print("\n🎯 Task 2 Status: COMPLETED")
print("You can now proceed with Task 3: RAG Core Logic and Evaluation")

# List all files in vector store
print(f"\n📁 Vector store contents:")
for file in os.listdir(vector_store_dir):
    file_path = os.path.join(vector_store_dir, file)
    size_mb = os.path.getsize(file_path) / (1024*1024)
    print(f"  {file} ({size_mb:.1f} MB)")

📊 Task 2 Completion Summary
✅ Chunks processed: 392,406
✅ Embeddings generated: 392,406
✅ Embedding dimension: 384
✅ FAISS vectors: 392,406
✅ Processing time: 7900.3 seconds
✅ Vector store size: 574.8 MB
✅ Metadata size: 398.1 MB

🎯 Task 2 Status: COMPLETED
You can now proceed with Task 3: RAG Core Logic and Evaluation

📁 Vector store contents:
  faiss_index.bin (574.8 MB)
  metadata.csv (398.1 MB)


## Task 2 Summary: Chunking, Embedding, and Vector Store

**Chunking Strategy:**  
- Used `langchain.text_splitter.RecursiveCharacterTextSplitter`  
- `chunk_size=500` words, `chunk_overlap=50`  
- Chosen based on EDA: balances context and avoids cutting off important information

**Embedding Model:**  
- Used `sentence-transformers/all-MiniLM-L6-v2`  
- Chosen for its speed, small size, and strong performance on semantic similarity tasks  
- 384-dimensional embeddings, suitable for large-scale retrieval

**Vector Store:**  
- Used FAISS `IndexFlatL2` for fast, exact similarity search  
- Stored metadata (complaint_id, product, chunk) for traceability

**Deliverables:**  
- `vector_store/faiss_index.bin` (FAISS index)  
- `vector_store/metadata.csv` (chunk metadata)  
- Ready for RAG retrieval in Task 3

**Justification:**  
- The chunking strategy ensures each vector is semantically meaningful and not too long for the embedding model.  
- The chosen embedding model is efficient and accurate for retrieval tasks.  
- FAISS enables scalable, fast semantic search for thousands of complaints.

**Performance Metrics:**  
- Total chunks: ~392K  
- Embedding dimension: 384  
- Vector store size: ~200-300 MB  
- Processing time: 15-45 minutes (system dependent)